# OpenVINO™ Model conversion

This notebook shows how to convert a model from original framework format to OpenVINO Intermediate Representation (IR).


#### Table of contents:

- [OpenVINO IR format](#OpenVINO-IR-format)
- [Fetching example models](#Fetching-example-models)
- [Conversion](#Conversion)
    - [Setting Input Shapes](#Setting-Input-Shapes)
    - [Compressing a Model to FP16](#Compressing-a-Model-to-FP16)
    - [Convert Models from memory](#Convert-Models-from-memory)
- [Migration from Legacy conversion API](#Migration-from-Legacy-conversion-API)
    - [Specifying Layout](#Specifying-Layout)
    - [Changing Model Layout](#Changing-Model-Layout)
    - [Specifying Mean and Scale Values](#Specifying-Mean-and-Scale-Values)
    - [Reversing Input Channels](#Reversing-Input-Channels)
    - [Cutting Off Parts of a Model](#Cutting-Off-Parts-of-a-Model)


### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/convert-to-openvino/convert-to-openvino.ipynb" />


In [1]:
# Required imports. Please execute this cell first.
%pip install -q "tensorflow-macos>=2.5; sys_platform == 'darwin' and platform_machine == 'arm64'"
%pip install -q "tensorflow>=2.5; sys_platform == 'darwin' and platform_machine != 'arm64'"
%pip install -q "tensorflow>=2.5; sys_platform != 'darwin'"
%pip install -q --no-deps "tensorflow-hub"
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu \
"openvino>=2024.4.0" "requests" "tqdm" "transformers>=4.31" "onnx!=1.16.2" "torch>=2.1" "torchvision" "tf_keras"

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: torch 2.4.1 has requirement typing-extensions>=4.8.0, but you'll have typing-extensions 4.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow<2.16,>=2.15 (from tf_keras) (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc

In [2]:
import requests
from pathlib import Path

if not Path("notebook_utils.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
    )
    open("notebook_utils.py", "w").write(r.text)

# Read more about telemetry collection at https://github.com/openvinotoolkit/openvino_notebooks?tab=readme-ov-file#-telemetry
from utils.notebook_utils import collect_telemetry

collect_telemetry("convert-to-openvino.ipynb")

## OpenVINO IR format
[back to top ⬆️](#Table-of-contents:)

OpenVINO [Intermediate Representation (IR)](https://docs.openvino.ai/2024/documentation/openvino-ir-format.html) is the proprietary model format of OpenVINO. It is produced after converting a model with model conversion API. Model conversion API translates the frequently used deep learning operations to their respective similar representation in OpenVINO and tunes them with the associated weights and biases from the trained model. The resulting IR contains two files: an `.xml` file, containing information about network topology, and a `.bin` file, containing the weights and biases binary data.

There are two ways to convert a model from the original framework format to OpenVINO IR: Python conversion API and OVC command-line tool. You can choose one of them based on whichever is most convenient for you.

OpenVINO conversion API supports next model formats: `PyTorch`, `TensorFlow`, `TensorFlow Lite`, `ONNX`, and `PaddlePaddle`. These model formats can be read, compiled, and converted to OpenVINO IR, either automatically or explicitly.

 For more details, refer to [Model Preparation](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html) documentation.

In [3]:
# OVC CLI tool parameters description

! ovc --help

/bin/bash: ovc: command not found


## Fetching example models
[back to top ⬆️](#Table-of-contents:)

This notebook uses two models for conversion examples:

* [Distilbert](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) NLP model from Hugging Face
* [Resnet50](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights) CV classification model from torchvision

In [ ]:
from pathlib import Path

# create a directory for models files
MODEL_DIRECTORY_PATH = Path("model")
MODEL_DIRECTORY_PATH.mkdir(exist_ok=True)

Fetch [distilbert](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) NLP model from Hugging Face and export it in ONNX format:

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

ONNX_NLP_MODEL_PATH = MODEL_DIRECTORY_PATH / "distilbert.onnx"

# download model
hf_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

if not ONNX_NLP_MODEL_PATH.exists():
    inputs = tokenizer("Hi, how are you?", return_tensors="pt")
    input_names = list(inputs.keys())
    dynamic_axes = {input_name: {0: "batch_size", 1: "seq_length"} for input_name in input_names}
    torch.onnx.export(
        hf_model, args=dict(inputs), input_names=input_names, output_names=["logits"], dynamic_axes=dynamic_axes, f=ONNX_NLP_MODEL_PATH, opset_version=14
    )
    print(f"ONNX model exported to {ONNX_NLP_MODEL_PATH}")

Fetch [Resnet50](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights) CV classification model from torchvision:

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
import torch
import warnings

# load model object
pytorch_model = model.load_state_dict(torch.load(PATH, weights_only=True))
# switch model from training to inference mode
pytorch_model.eval()


model.eval()

Convert PyTorch model to ONNX format:

In [6]:
from ultralytics import YOLO

model = YOLO("yolov3-tiny.pt")
# Export the model to ONNX format
model.export(format="onnx")  

PRO TIP 💡 Replace 'model=yolov3-tiny.pt' with new 'model=yolov3-tinyu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 23.3M/23.3M [00:03<00:00, 6.75MB/s]


Ultralytics 8.3.36 🚀 Python-3.8.10 torch-2.4.1+cu121 CPU (AMD Ryzen 7 5800H with Radeon Graphics)
YOLOv3-tiny summary (fused): 63 layers, 12,168,784 parameters, 0 gradients, 19.0 GFLOPs

PyTorch: starting from 'yolov3-tinyu.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 2000) (23.3 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.42...
ONNX: export success ✅ 2.3s, saved as 'yolov3-tinyu.onnx' (46.5 MB)

Export complete (2.9s)
Results saved to /home/sonieth/testing_neural_stick2/ncs2_test/prepare
Predict:         yolo predict task=detect model=yolov3-tinyu.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov3-tinyu.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov3-tinyu.onnx'

In [5]:

ONNX_CV_MODEL_PATH =  "/home/sonieth/testing_neural_stick2/ncs2_test/prepare/yolov3-tiny.onnx" #MODEL_DIRECTORY_PATH / "resnet.onnx"
PT_MODEL_PATH = "/home/sonieth/testing_neural_stick2/ncs2_test/prepare/yolov3-tiny.pt"
# if ONNX_CV_MODEL_PATH.exists():
#     print(f"ONNX model {ONNX_CV_MODEL_PATH} already exists.")
# else:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    torch.onnx.export(model=PT_MODEL_PATH, args=torch.randn(1, 3, 224, 224), f=ONNX_CV_MODEL_PATH)
print(f"ONNX model exported to {ONNX_CV_MODEL_PATH}")

AttributeError: 'str' object has no attribute 'modules'

## Conversion
[back to top ⬆️](#Table-of-contents:)

To convert a model to OpenVINO IR, use the following API:

In [9]:
import openvino as ov

ONNX_NLP_MODEL_PATH = "yolov3-tinyu.onnx"
MODEL_DIRECTORY_PATH = "/home/sonieth/testing_neural_stick2/ncs2_test/prepare"
# ov.convert_model returns an openvino.runtime.Model object
print(ONNX_NLP_MODEL_PATH)
ov_model = ov.convert_model(ONNX_NLP_MODEL_PATH)


# then model can be serialized to *.xml & *.bin files
ov.save_model(ov_model, MODEL_DIRECTORY_PATH / "distilbert.xml")


yolov3-tinyu.onnx


AttributeError: module 'openvino' has no attribute 'convert_model'

In [ ]:
! ovc model/distilbert.onnx --output_model model/distilbert.xml

#### Setting Input Shapes
[back to top ⬆️](#Table-of-contents:)

Model conversion is supported for models with dynamic input shapes that contain undefined dimensions. However, if the shape of data is not going to change from one inference request to another, it is recommended to set up static shapes (when all dimensions are fully defined) for the inputs. Doing so at the model preparation stage, not at runtime, can be beneficial in terms of performance and memory consumption.

For more information refer to [Setting Input Shapes](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/setting-input-shapes.html) documentation.

In [ ]:
import openvino as ov

ov_model = ov.convert_model(ONNX_NLP_MODEL_PATH, input=[("input_ids", [1, 128]), ("attention_mask", [1, 128])])

In [ ]:
! ovc model/distilbert.onnx --input input_ids[1,128],attention_mask[1,128] --output_model model/distilbert.xml

The `input` parameter allows overriding original input shapes if it is supported by the model topology. Shapes with dynamic dimensions in the original model can be replaced with static shapes for the converted model, and vice versa. The dynamic dimension can be marked in model conversion API parameter as `-1` or `?` when using `ovc`:

In [ ]:
import openvino as ov

ov_model = ov.convert_model(ONNX_NLP_MODEL_PATH, input=[("input_ids", [1, -1]), ("attention_mask", [1, -1])])

In [ ]:
! ovc model/distilbert.onnx --input "input_ids[1,?],attention_mask[1,?]" --output_model model/distilbert.xml

To optimize memory consumption for models with undefined dimensions in runtime, model conversion API provides the capability to define boundaries of dimensions. The boundaries of undefined dimension can be specified with ellipsis in the command line or with `openvino.Dimension` class in Python. For example, launch model conversion for the ONNX Bert model and specify a boundary for the sequence length dimension:

In [ ]:
import openvino as ov


sequence_length_dim = ov.Dimension(10, 128)

ov_model = ov.convert_model(
    ONNX_NLP_MODEL_PATH,
    input=[
        ("input_ids", [1, sequence_length_dim]),
        ("attention_mask", [1, sequence_length_dim]),
    ],
)

In [ ]:
! ovc model/distilbert.onnx --input input_ids[1,10..128],attention_mask[1,10..128] --output_model model/distilbert.xml

#### Compressing a Model to FP16
[back to top ⬆️](#Table-of-contents:)

By default model weights compressed to FP16 format when saving OpenVINO model to IR. This saves up to 2x storage space for the model file and in most cases doesn't sacrifice model accuracy. Weight compression can be disabled by setting `compress_to_fp16` flag to `False`:

In [ ]:
import openvino as ov

ov_model = ov.convert_model(ONNX_NLP_MODEL_PATH)
ov.save_model(ov_model, MODEL_DIRECTORY_PATH / "distilbert.xml", compress_to_fp16=False)

In [ ]:
! ovc model/distilbert.onnx --output_model model/distilbert.xml --compress_to_fp16=False

#### Convert Models from memory
[back to top ⬆️](#Table-of-contents:)

Model conversion API supports passing original framework Python object directly. More details can be found in [PyTorch](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/convert-model-pytorch.html), [TensorFlow](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/convert-model-tensorflow.html), [PaddlePaddle](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/convert-model-paddle.html) frameworks conversion guides.

In [ ]:
import openvino as ov
import torch

example_input = torch.rand(1, 3, 224, 224)

ov_model = ov.convert_model(pytorch_model, example_input=example_input, input=example_input.shape)

In [ ]:
import os

import openvino as ov
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = str(Path("./tfhub_modules").resolve())

model = hub.load("https://www.kaggle.com/models/google/movenet/frameworks/TensorFlow2/variations/singlepose-lightning/versions/4")
movenet = model.signatures["serving_default"]

ov_model = ov.convert_model(movenet)

## Migration from Legacy conversion API
[back to top ⬆️](#Table-of-contents:)

In the 2023.1 OpenVINO release OpenVINO Model Conversion API was introduced with the corresponding Python API: `openvino.convert_model` method. `ovc` and `openvino.convert_model` represent a lightweight alternative of `mo` and `openvino.tools.mo.convert_model` which are considered legacy API now.
`mo.convert_model()` provides a wide range of preprocessing parameters. Most of these parameters have analogs in OVC or can be replaced with functionality from `ov.PrePostProcessor` class. Refer to [Optimize Preprocessing notebook](../optimize-preprocessing/optimize-preprocessing.ipynb) for more information about [Preprocessing API](https://docs.openvino.ai/2024/openvino-workflow/running-inference/optimize-inference/optimize-preprocessing.html). Here is the migration guide from legacy model preprocessing to Preprocessing API.

#### Specifying Layout
[back to top ⬆️](#Table-of-contents:)

Layout defines the meaning of dimensions in a shape and can be specified for both inputs and outputs. Some preprocessing requires to set input layouts, for example, setting a batch, applying mean or scales, and reversing input channels (BGR<->RGB). For the layout syntax, check the [Layout API overview](https://docs.openvino.ai/2024/openvino-workflow/running-inference/optimize-inference/optimize-preprocessing/layout-api-overview.html). To specify the layout, you can use the layout option followed by the layout value.

The following example specifies the `NCHW` layout for a Pytorch Resnet50 model that was exported to the ONNX format:

In [ ]:
# Converter API
import openvino as ov

ov_model = ov.convert_model(ONNX_CV_MODEL_PATH)

prep = ov.preprocess.PrePostProcessor(ov_model)
prep.input("input.1").model().set_layout(ov.Layout("nchw"))
ov_model = prep.build()

```python
# Legacy Model Optimizer API
from openvino.tools import mo

ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, layout="nchw")
```

#### Changing Model Layout
[back to top ⬆️](#Table-of-contents:)

Transposing of matrices/tensors is a typical operation in Deep Learning - you may have a BMP image `640x480`, which is an array of `{480, 640, 3}` elements, but Deep Learning model can require input with shape `{1, 3, 480, 640}`.

Conversion can be done implicitly, using the layout of a user’s tensor and the layout of an original model:

In [ ]:
# Converter API
import openvino as ov

ov_model = ov.convert_model(ONNX_CV_MODEL_PATH)

prep = ov.preprocess.PrePostProcessor(ov_model)
prep.input("input.1").tensor().set_layout(ov.Layout("nhwc"))
prep.input("input.1").model().set_layout(ov.Layout("nchw"))
ov_model = prep.build()

# Legacy Model Optimizer API
```python
from openvino.tools import mo

ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, layout="nchw->nhwc")

# alternatively use source_layout and target_layout parameters
ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, source_layout="nchw", target_layout="nhwc")
```

#### Specifying Mean and Scale Values
[back to top ⬆️](#Table-of-contents:)

Using Preprocessing API `mean` and `scale` values can be set. Using these API, model embeds the corresponding preprocessing block for mean-value normalization of the input data and optimizes this block. Refer to [Optimize Preprocessing notebook](../optimize-preprocessing/optimize-preprocessing.ipynb) for more examples.

In [ ]:
# Converter API
import openvino as ov

ov_model = ov.convert_model(ONNX_CV_MODEL_PATH)

prep = ov.preprocess.PrePostProcessor(ov_model)
prep.input("input.1").tensor().set_layout(ov.Layout("nchw"))
prep.input("input.1").preprocess().mean([255 * x for x in [0.485, 0.456, 0.406]])
prep.input("input.1").preprocess().scale([255 * x for x in [0.229, 0.224, 0.225]])

ov_model = prep.build()

```python
# Legacy Model Optimizer API

from openvino.tools import mo


ov_model = mo.convert_model(
    ONNX_CV_MODEL_PATH,
    mean_values=[255 * x for x in [0.485, 0.456, 0.406]],
    scale_values=[255 * x for x in [0.229, 0.224, 0.225]],
)
```

#### Reversing Input Channels
[back to top ⬆️](#Table-of-contents:)

Sometimes, input images for your application can be of the `RGB` (or `BGR`) format, and the model is trained on images of the `BGR` (or `RGB`) format, which is in the opposite order of color channels. In this case, it is important to preprocess the input images by reverting the color channels before inference.

In [ ]:
# Converter API
import openvino as ov

ov_model = ov.convert_model(ONNX_CV_MODEL_PATH)

prep = ov.preprocess.PrePostProcessor(ov_model)
prep.input("input.1").tensor().set_layout(ov.Layout("nchw"))
prep.input("input.1").preprocess().reverse_channels()
ov_model = prep.build()

```python
# Legacy Model Optimizer API
from openvino.tools import mo

ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, reverse_input_channels=True)
```

#### Cutting Off Parts of a Model
[back to top ⬆️](#Table-of-contents:)

Cutting model inputs and outputs from a model is no longer available in the new conversion API. Instead, we recommend performing the cut in the original framework. Examples of model cutting of TensorFlow protobuf, TensorFlow SavedModel, and ONNX formats with tools provided by the Tensorflow and ONNX frameworks can be found in [documentation guide](https://docs.openvino.ai/2024/documentation/legacy-features/transition-legacy-conversion-api.html#cutting-off-parts-of-a-model). For PyTorch, TensorFlow 2 Keras, and PaddlePaddle, we recommend changing the original model code to perform the model cut.